In [1]:
import dagshub
import mlflow
from mlflow import MlflowClient

In [2]:
dagshub.init(repo_owner='zapatacc', repo_name='final-exam-pcd2024-autumn', mlflow=True)
mlflow.set_experiment("jesus-carbajal-logreg-label-encoder")
TRACKING_URI = mlflow.get_tracking_uri()

Accessing as Pepe-Chuy

Initialized MLflow to track repo "zapatacc/final-exam-pcd2024-autumn"

Repository zapatacc/final-exam-pcd2024-autumn initialized!

In [41]:
import mlflow

mlflow.set_experiment("jesus-carbajal-logreg-label-encoder")
client = MlflowClient()

# Get the experiment ID
experiment = client.get_experiment_by_name("jesus-carbajal-logreg-label-encoder")
experiment_id = experiment.experiment_id

#Get 100 experiments
runs = client.search_runs(
    experiment_ids=[experiment_id],
    run_view_type=mlflow.entities.ViewType.ACTIVE_ONLY,
    max_results=100,
    order_by=["start_time desc"]
)

best_run = 0
best_accuracy = 0

for run in runs:
    accuracy = run.data.metrics.get('accuracy', None)
    if accuracy is not None and accuracy > best_accuracy:
        best_accuracy = accuracy
        best_run = run

if best_run:
    print(f"Best Run ID: {best_run.info.run_id}")
    print(f"Accuracy: {best_accuracy}")


model_uri = f"runs:/{best_run.info.run_id}/model"
model_name = "jesus-carbajal-model"
registered_model = mlflow.register_model(model_uri=model_uri, name=model_name)

model_version = client.get_latest_versions(name=model_name, stages=["None"])[0].version
client.set_registered_model_alias(
        name=model_name,
        alias="Champ",
        version=model_version
    )

Best Run ID: 8ccf9f0120494d78a04c99aa0b113d82
Accuracy: 0.5718428684418666


Registered model 'jesus-carbajal-model' already exists. Creating a new version of this model...
2024/11/21 02:36:26 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: jesus-carbajal-model, version 2
Created version '2' of model 'jesus-carbajal-model'.
/tmp/ipykernel_345485/302226163.py:36: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  model_version = client.get_latest_versions(name=model_name, stages=["None"])[0].version


In [3]:
run_uri = 'runs:/8ccf9f0120494d78a04c99aa0b113d82/pipeline_model'


In [4]:
dv = mlflow.pyfunc.load_model(run_uri)

/home/pepechuy/Downloads/School/Lab5/Examen/final-exam-pcd2024-autumn/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
dv.predict(["Runner coneda olerd"])

array(['Credit card or prepaid card + General-purpose credit card or charge card'],
      dtype=object)

In [48]:
dagshub.init(repo_owner='zapatacc', repo_name='final-exam-pcd2024-autumn', mlflow=True)

mlflow.set_experiment("jesus-carbajal-logreg-label-encoder")
TRACKING_URI = mlflow.get_tracking_uri()

run_uri = 'runs:/8ccf9f0120494d78a04c99aa0b113d82/pipeline_model'

dv = mlflow.pyfunc.load_model(run_uri)

Initialized MLflow to track repo "zapatacc/final-exam-pcd2024-autumn"

Repository zapatacc/final-exam-pcd2024-autumn initialized!

In [7]:

dagshub_repo = "https://dagshub.com/zapatacc/final-exam-pcd2024-autumn"
MLFLOW_TRACKING_URI = "https://dagshub.com/zapatacc/final-exam-pcd2024-autumn.mlflow"
mlflow.set_tracking_uri(uri=MLFLOW_TRACKING_URI)
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

run_uri = 'runs:/8ccf9f0120494d78a04c99aa0b113d82/pipeline_model'

dv = mlflow.pyfunc.load_model(run_uri)

In [ ]:

# dagshub.init(repo_owner='zapatacc', repo_name='final-exam-pcd2024-autumn', mlflow=True)
# mlflow.set_experiment("jesus-carbajal-logreg-label-encoder")

# TRACKING_URI = mlflow.get_tracking_uri()

# run_uri = 'runs:/8ccf9f0120494d78a04c99aa0b113d82/pipeline_model'

# dv = mlflow.pyfunc.load_model(run_uri)



dagshub_repo = "https://dagshub.com/zapatacc/final-exam-pcd2024-autumn"
MLFLOW_TRACKING_URI = "https://dagshub.com/zapatacc/final-exam-pcd2024-autumn.mlflow"
mlflow.set_tracking_uri(uri=MLFLOW_TRACKING_URI)
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

run_ = mlflow.search_runs(order_by=['metrics.accuracy'],
                          output_format="list",
                          experiment_names=["pipeline_model"]
                          )[0]

run_id = run_.info.run_id

run_uri = f"runs:/{run_id}/preprocessor"

client.download_artifacts(
    run_id=run_id,
    path='preprocessor',
    dst_path='.'
)

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

model_name = "nyc-taxi-model"
alias = "champion"

model_uri = f"models:/{model_name}@{alias}"

champion_model = mlflow.pyfunc.load_model(
    model_uri=model_uri
)

In [8]:
dagshub_repo = "https://dagshub.com/zapatacc/final-exam-pcd2024-autumn"
MLFLOW_TRACKING_URI = "https://dagshub.com/zapatacc/final-exam-pcd2024-autumn.mlflow"
mlflow.set_tracking_uri(uri=MLFLOW_TRACKING_URI)
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

run_uri = 'runs:/8ccf9f0120494d78a04c99aa0b113d82/pipeline_model'

dv = mlflow.pyfunc.load_model(run_uri)


In [12]:
dv.predict([' yfuviu undfninidf nfiunudi ijainsdi iidi'])

array(['Credit card or prepaid card + General-purpose credit card or charge card'],
      dtype=object)

In [7]:
import mlflow
import dagshub

In [10]:
dagshub.init(repo_owner='zapatacc', repo_name='final-exam-pcd2024-autumn', mlflow=True)
mlflow.set_experiment("jesus-carbajal-logreg")

def getChamp():
    mlflow.set_experiment("jesus-carbajal-logreg-label-encoder")
    client = MlflowClient()

    # Get the experiment ID
    experiment = client.get_experiment_by_name("jesus-carbajal-logreg-label-encoder")
    experiment_id = experiment.experiment_id

    #Get 100 experiments
    runs = client.search_runs(
        experiment_ids=[experiment_id],
        run_view_type=mlflow.entities.ViewType.ACTIVE_ONLY,
        max_results=100,
        order_by=["start_time desc"]
    )

    best_run = 0
    best_accuracy = 0

    for run in runs:
        accuracy = run.data.metrics.get('accuracy', None)
        if accuracy is not None and accuracy > best_accuracy:
            best_accuracy = accuracy
            best_run = run

    return best_run.info.run_id

def setChamp(run_id, model_name):
    
    client = MlflowClient()
    model_uri = f"runs:/{run_id}/model"
    registered_model = mlflow.register_model(model_uri=model_uri, name=model_name)

    model_version = client.get_latest_versions(name=model_name, stages=["None"])[0].version
    client.set_registered_model_alias(
        name=model_name,
        alias="Champ",
        version=model_version
    )

champ = getChamp()
setChamp(champ, "jesus-carbajal-model")

Initialized MLflow to track repo "zapatacc/final-exam-pcd2024-autumn"

Repository zapatacc/final-exam-pcd2024-autumn initialized!

Registered model 'jesus-carbajal-model' already exists. Creating a new version of this model...
2024/11/21 19:23:58 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: jesus-carbajal-model, version 4
Created version '4' of model 'jesus-carbajal-model'.
/tmp/ipykernel_17859/2768569986.py:37: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  model_version = client.get_latest_versions(name=model_name, stages=["None"])[0].version


In [9]:
import mlflow
from mlflow.tracking import MlflowClient

# Initialize DagsHub and set experiment
dagshub.init(repo_owner='zapatacc', repo_name='final-exam-pcd2024-autumn', mlflow=True)
mlflow.set_experiment("jesus-carbajal-logreg")

def getChamp():
    """
    Finds the best model run based on the highest accuracy metric 
    in the 'jesus-carbajal-logreg-label-encoder' experiment.
    """
    mlflow.set_experiment("jesus-carbajal-logreg-label-encoder")
    client = MlflowClient()

    # Get the experiment ID
    experiment = client.get_experiment_by_name("jesus-carbajal-logreg-label-encoder")
    experiment_id = experiment.experiment_id

    # Get up to 100 experiment runs sorted by start time
    runs = client.search_runs(
        experiment_ids=[experiment_id],
        run_view_type=mlflow.entities.ViewType.ACTIVE_ONLY,
        max_results=100,
        order_by=["start_time desc"]
    )

    best_run = None
    best_accuracy = 0

    # Identify the run with the highest accuracy
    for run in runs:
        accuracy = run.data.metrics.get('accuracy', None)
        if accuracy is not None and accuracy > best_accuracy:
            best_accuracy = accuracy
            best_run = run

    return best_run.info.run_id if best_run else None

def setChamp(run_id, model_name):
    """
    Registers the given model as the 'Champ' version for the specified model name.
    """
    client = MlflowClient()
    model_uri = f"runs:/{run_id}/model"
    registered_model = mlflow.register_model(model_uri=model_uri, name=model_name)

    # Get the latest version of the model and set the alias to "Champ"
    model_version = client.get_latest_versions(name=model_name, stages=["None"])[0].version
    client.set_registered_model_alias(
        name=model_name,
        alias="Champ",
        version=model_version
    )

# Main execution
champ = getChamp()
if champ:
    setChamp(champ, "jesus-carbajal-model")
else:
    print("No suitable champion model found.")


Initialized MLflow to track repo "zapatacc/final-exam-pcd2024-autumn"

Repository zapatacc/final-exam-pcd2024-autumn initialized!

Registered model 'jesus-carbajal-model' already exists. Creating a new version of this model...
2024/11/21 19:22:43 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: jesus-carbajal-model, version 3
Created version '3' of model 'jesus-carbajal-model'.
/tmp/ipykernel_17859/380569046.py:49: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  model_version = client.get_latest_versions(name=model_name, stages=["None"])[0].version
